In [1]:
import pandas as pd
import numpy as np
import utils.preprocess_funcs as pf

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dariasatco/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
%load_ext autoreload
%autoreload 2

### What is the right price to set up for your listing?

Here we consider the following case: you decided to add an object (room/appartment) to airbnb. You are new on this platform and in this business, so you are not sure what is the right price to set up. You fill all the information about the place and on the stage of putting the price you can get an advice from the system, which is based on the current proposition in your city/neighbourhood. 

Let us build a model, which will be recommend a suitable price based on your listing characteristics.

In [3]:
# select columns for modelling
numerical_columns = ['host_listings_count', 'accommodates', 'bathrooms',
                     'bedrooms', 'beds', 'square_feet', 'guests_included',
                     'minimum_nights', 'maximum_nights', 'density', 
                     'district_sentiment', 'district_rev_score']

object_columns = ['room_type', 'property_type', 'bed_type', 'instant_bookable']
inc_list_col = ['amenities']
target = ['price']

cols_to_take = numerical_columns + object_columns + inc_list_col + target

# filters on listing activity 
LAST_REVIEW_DAYS = 180
LAST_CALENDAR_UPD = 60

Read data and preprocess categorical columns

In [8]:
dataset = pd.read_pickle('Boston_Airbnb_data/preprocessed_listings.pickle')
dataset = dataset[ (dataset['recently_booked']==1) 
                & (dataset['recently_calendar_upd']==1) ]
dataset = dataset[cols_to_take]

In [9]:
# preprocess categorical columns
preprocessed = pf.add_dummy_cols(dataset, cat_cols=object_columns, dummy_na=True)
preprocessed = pf.one_hot_enc_for_amenities(preprocessed)
# fill all NaN values with -1
preprocessed = preprocessed.fillna(-1)

Build model